In [1]:
# Get some libs in your life
import sys, os, time
import geopandas as gpd
import pandas as pd
import rasterio as rt
import numpy as np
from affine import Affine
from shapely.geometry import Point, LineString
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import box
from shapely.wkt import loads
from rasterio import features
import networkx as nx
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets')
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST')
import GOSTnet as gn
import importlib

peartree version: 0.6.0 
networkx version: 2.2 
matplotlib version: 2.2.2 
osmnx version: 0.8.2 


### Initial Graph Read In

In [2]:
bpth = r'C:\Users\charl\Documents\GOST\Yemen\graphtool'
G = nx.read_gpickle(os.path.join(bpth, 'conflict_adj.pickle'))

### Walk all tiles, find path

In [3]:
pth = r'C:\Users\charl\Documents\GOST\Yemen\SRTM\high_res'
tiles = []
for root, folder, files in os.walk(pth):
    for f in files:
        if f[-3:] == 'hgt':
            tiles.append(f[:-4])

### Load dictionary of tiles 

In [4]:
arrs = {}
for t in tiles:
    fpath = r'C:\Users\charl\Documents\GOST\Yemen\SRTM\high_res\{}.hgt\{}.hgt'.format(t, t)
    arrs[t] = rt.open(fpath, 'r')

### Add correct tile for each node into Graph Data Dictionary

In [5]:
uniques = []
for u, data in G.nodes(data = True):
    E = str(data['x'])[:2]
    N = str(data['y'])[:2]
    data['code'] = 'N{}E0{}'.format(N, E)
    uniques.append('N{}E0{}'.format(N, E))
unique_codes = list(set(uniques))

### Match on High Precision Elevation

In [6]:
property_name = 'elevation'
for code in unique_codes:
    list_of_nodes = {}
    for u, data in G.nodes(data=True):
        if data['code'] == code:
            list_of_nodes.update({u:(data['x'], data['y'])})
    dataset = arrs[code]
    b = dataset.bounds
    datasetBoundary = box(b[0], b[1], b[2], b[3])
    selKeys = []
    selPts = []
    for key, pt in list_of_nodes.items():
        if Point(pt[0], pt[1]).intersects(datasetBoundary):
            selPts.append(pt)
            selKeys.append(key)
    raster_values = list(dataset.sample(selPts))
    raster_values = [x[0] for x in raster_values]

    # generate new dictionary of {node ID: raster values}
    ref = dict(zip(selKeys, raster_values))
    for u, data in G.nodes(data=True):
        if u in ref.keys():
            data[property_name] = ref[u]

### Add on low precision elevation for missed nodes

In [7]:
for u, data in G.nodes(data=True):
    if data['elevation'] < -50:
        list_of_nodes.update({u:(data['x'], data['y'])})
        
tifpath = r'C:\Users\charl\Documents\GOST\Yemen\SRTM\clipped'
t = r'clipped_e20N40.tif'
tt = os.path.join(tifpath, t)
dataset = rt.open(tt, 'r')
b = dataset.bounds
datasetBoundary = box(b[0], b[1], b[2], b[3])
selKeys = []
selPts = []
for key, pt in list_of_nodes.items():
    if Point(pt[0], pt[1]).intersects(datasetBoundary):
        selPts.append(pt)
        selKeys.append(key)
raster_values = list(dataset.sample(selPts))
raster_values = [x[0] for x in raster_values]

# generate new dictionary of {node ID: raster values}
ref = dict(zip(selKeys, raster_values))
for u, data in G.nodes(data=True):
    if u in ref.keys():
        data['elevation'] = ref[u]

### For the remaining mistakes, set elevation to 0

In [8]:
for u, data in G.nodes(data=True):
    if data['elevation'] < 0:
        data['elevation'] = 0

### Generate dictionary of elevations for each node

In [9]:
elev_dict = {}
for u, data in G.nodes(data = True):
    if 'elevation' in data.keys(): 
        if data['elevation'] < 0:
            elev_dict[u] = 0
        else:
            elev_dict[u] = data['elevation']
    else:
        elev_dict[u] = 0

In [10]:
# Sense check - max and minimum elevationr recorded
max(list(elev_dict.values())), min(list(elev_dict.values()))

(3652, 0)

### Generate New Walking Data in existing edges

In [11]:
# Tobler's hiking function: https://en.wikipedia.org/wiki/Tobler%27s_hiking_function
def speed(incline_ratio, max_speed):
    walkspeed = max_speed * np.exp(-3.5 * abs(incline_ratio + 0.05)) 
    return walkspeed

max_walkspeed = 6
min_speed = 0.1

for u, v, data in G.edges(data = True):
    data['elev_start'] = elev_dict[u]
    data['elev_end'] = elev_dict[v]
    if data['length'] == 0:
        data['walkspeed'] = 0
        data['walk_time'] = 0
    else:
        delta_elevation = data['elev_end'] - data['elev_start']
        incline_ratio = delta_elevation / data['length']
        speed_kmph = speed(incline_ratio = incline_ratio, max_speed = max_walkspeed)
        speed_kmph = max(speed_kmph, min_speed)
        data['walkspeed'] = speed_kmph
        data['walk_time'] = data['length'] / 1000 * 3600 / speed_kmph

### Save Down

In [13]:
gn.example_edge(G)

(0, 40443, {'Wkt': 'LINESTRING (44.0580736 13.5904793, 44.058066 13.5904398)', 'id': 29471, 'infra_type': 'track', 'osm_id': 117965664, 'country': 'YEM', 'key': 'edge_29471', 'length': 4.446831582994608, 'Type': 'legitimate', 'time': 1.0672395799187058, 'mode': 'drive', 'ID': 1, 'time_November1st': 1.0672395799187058, 'MOD_November1st': 'normal', 'time_November8th': 1.0672395799187058, 'MOD_November8th': 'normal', 'time_November14th': 1.0672395799187058, 'MOD_November14th': 'normal', 'time_November25th': 1.0672395799187058, 'MOD_November25th': 'normal', 'time_December17th': 1.0672395799187058, 'MOD_December17th': 'normal', 'time_January24th': 1.0672395799187058, 'MOD_January24th': 'normal', 'elev_start': 1291, 'elev_end': 1291, 'walkspeed': 5.036742124615245, 'walk_time': 3.1783627794928013})


In [12]:
gn.save(G, 'walk_graph', bpth)